In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load Fashion MNIST dataset
def load_data(batch_size=32):
    """
    Load Fashion MNIST dataset and create train/test splits.
    
    Parameters:
        batch_size (int): Batch size for data loader.
    
    Returns:
        tuple: DataLoaders for training and validation.
    """
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    
    dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, val_loader

# Define CNN Model for feature extraction
class FashionCNN(nn.Module):
    """
    Convolutional Neural Network (CNN) for extracting clothing features.
    """
    def __init__(self):
        super(FashionCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)  # Feature embedding
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        feature_vector = self.relu(self.fc1(x))  # Extracted features
        return feature_vector

# Train the CNN model
def train_model(model, train_loader, epochs=10, lr=0.001):
    """
    Train the CNN model for feature extraction.
    
    Parameters:
        model (nn.Module): CNN model.
        train_loader (DataLoader): Training data loader.
        epochs (int): Number of epochs.
        lr (float): Learning rate.
    """
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    model.train()
    
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")

# Compute Cosine Similarity
def compute_similarity(model, img1, img2):
    """
    Compute cosine similarity between two clothing images.
    
    Parameters:
        model (nn.Module): Trained CNN model.
        img1 (Tensor): First clothing image.
        img2 (Tensor): Second clothing image.
    
    Returns:
        float: Cosine similarity score.
    """
    model.eval()
    with torch.no_grad():
        feature1 = model(img1.unsqueeze(0))
        feature2 = model(img2.unsqueeze(0))
        similarity = cosine_similarity(feature1.cpu().numpy(), feature2.cpu().numpy())
    return similarity[0][0]

# Example execution
train_loader, val_loader = load_data()

# Initialize and train model
model = FashionCNN()
train_model(model, train_loader, epochs=10)

# Example similarity check (using first two images from validation set)
data_iter = iter(val_loader)
images, _ = next(data_iter)
similarity_score = compute_similarity(model, images[0], images[1])
print(f"Similarity Score: {similarity_score:.4f}")
